In [1]:
# import from the files in this directory
from dotenv import load_dotenv
from classifier import *
from data_fetcher import *
from parser import *
from orchestrator import *
from logger_setup import *
import os
import json
import pandas as pd
import numpy as np
import re
import time

In [2]:
config_path = 'config_experiment.json'  # Config with input file details
load_dotenv()
orchestrator = Orchestrator(config_path)

orchestrator.py - line 20 - INFO - Data_Gatherer Orchestrator initialized. Extraction step Model: gemini-2.0-flash-exp


In [3]:
urls = None
df = pd.read_csv('exp_input/proteomexchange_search.tsv', sep='\t')
publication_datasets = df[['publication','identifier','repository']]

In [4]:
# get an idea of values in df publication
publication_datasets['publication'].value_counts()

publication
Dataset with its publication pending                                                                                                                                                                      11719
no publication                                                                                                                                                                                             2550
<a href="http://www.ncbi.nlm.nih.gov/pubmed/35084980" target="_blank">Melani et al. (2022)</a>                                                                                                               56
<a href="http://www.ncbi.nlm.nih.gov/pubmed/28267743" target="_blank">Matsumoto et al. (2017)</a>                                                                                                            28
<a href="http://www.ncbi.nlm.nih.gov/pubmed/28071820" target="_blank">Kreutz et al. (2017)</a>                                                              

In [5]:
#  clean the `publication` column by filtering out unwanted values like `"Dataset with its publication pending"`, `"no publication"`, and any HTML tags. 
# Remove rows with unwanted values
filtered_df = publication_datasets[~publication_datasets['publication'].isin(["Dataset with its publication pending", "no publication"])]
filtered_df[381:]

,publication,identifier,repository
899,"<a href=""https://dx.doi.org/10.1101/2024.09.18...",PXD054415,PRIDE
901,"<a href=""https://www.ncbi.nlm.nih.gov/pubmed/3...",PXD029492,PRIDE
902,"<a href=""https://www.ncbi.nlm.nih.gov/pubmed/3...",PXD054117,PRIDE
906,"<a href=""https://dx.doi.org/10.1186/S12964-024...",PXD051580,PRIDE
907,"<a href=""https://dx.doi.org/10.1038/S42004-024...",PXD055488,PRIDE
...,...,...,...
40130,"<a href=""http://www.ncbi.nlm.nih.gov/pubmed/23...",PXD000056,PRIDE
40131,"<a href=""http://www.ncbi.nlm.nih.gov/pubmed/22...",PXD000009,PRIDE
40133,"<a href=""http://www.ncbi.nlm.nih.gov/pubmed/22...",PXD000012,PRIDE
40134,"<a href=""http://www.ncbi.nlm.nih.gov/pubmed/22...",PXD000013,PRIDE


In [6]:
# Assuming filtered_df is already defined
filtered_df['publication_link'] = None  # Create a new column for the links

for i, row in filtered_df.iterrows():
    pub = str(row['publication'])  # Ensure string type
    if "href" in pub:
        match = re.findall(r'href=[\'"]([^\'"]+)[\'"]', pub)
        if match:
            filtered_df.at[i, 'publication_link'] = match  # Assign all the matched URLs
        else:
            filtered_df.at[i, 'publication_link'] = None
    else:
        filtered_df.at[i, 'publication_link'] = None

# Drop rows with missing links (optional)
filtered_df.dropna(subset=['publication_link'], inplace=True)
filtered_df.reset_index(drop=True, inplace=True)

/var/folders/br/873byd752sb72sf4n7yfykxw0000gn/T/ipykernel_28083/1978044078.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['publication_link'] = None  # Create a new column for the links
/var/folders/br/873byd752sb72sf4n7yfykxw0000gn/T/ipykernel_28083/1978044078.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.dropna(subset=['publication_link'], inplace=True)


In [7]:
m = 0
for i,row in filtered_df.iterrows():
    id = row['identifier']
    m+=len(row['publication_link'])
print(m/i+1)
print(filtered_df)

2.5313095933060907
                                             publication identifier  \
0      <a href="https://dx.doi.org/10.1038/S41467-025...  PXD059466   
1      <a href="https://dx.doi.org/10.6019/PXD051312"...  PXD051312   
2      <a href="https://dx.doi.org/10.17159/SAJS.2025...  PXD054431   
3      <a href="https://dx.doi.org/10.1101/2024.04.03...  PXD050443   
4      <a href="https://dx.doi.org/10.1101/2025.02.05...  PXD051704   
...                                                  ...        ...   
25691  <a href="http://www.ncbi.nlm.nih.gov/pubmed/23...  PXD000056   
25692  <a href="http://www.ncbi.nlm.nih.gov/pubmed/22...  PXD000009   
25693  <a href="http://www.ncbi.nlm.nih.gov/pubmed/22...  PXD000012   
25694  <a href="http://www.ncbi.nlm.nih.gov/pubmed/22...  PXD000013   
25695  <a href="http://www.ncbi.nlm.nih.gov/pubmed/22...  PXD000003   

         repository                                   publication_link  
0             PRIDE    [https://dx.doi.org/10.1038/S414

In [8]:
data = []

start = 441

iter_max = 50

t0 = time.time()

for i,row in filtered_df[start:].iterrows():
    
    driver = orchestrator.setup_data_fetcher()
        
    id = row['identifier']
    
    if i == 441 + iter_max:
        break
        
    if i%100 == 0 and i>0:
        print(f"Progress {i+1}/{len(filtered_df)}. ETA {((time.time()-t0)/(i+1))*(len(filtered_df)-i-1)}")
        
    print(f"Processing URL {i+1}.\nTime elapsed: {time.time()-t0}") if i>0 else None
    print(f"{len(row['publication_link'])} links found for dataset {id}")
    

    for url in row['publication_link']:
        orchestrator.logger.info(f"Processing URL: {url}")
    
        orchestrator.current_url = url
        orchestrator.publisher = orchestrator.data_fetcher.url_to_publisher_domain(url)
        
        orchestrator.data_fetcher = orchestrator.data_fetcher.update_DataFetcher_settings(url, orchestrator.full_DOM, orchestrator.logger)
        
        try:
            orchestrator.logger.info("Fetching Raw content")
            raw_data = orchestrator.data_fetcher.fetch_data(url)
            if id in raw_data:
                data.append({"publication": url,"dataset_uid": row['identifier'], "repo_name": row['repository'], "raw_html": raw_data})
                break
            else:
                print("id not found in raw data")
                continue
        except Exception as e:
            orchestrator.logger.error(f"Error processing URL {url}: {e}", exc_info=True)
    
    driver.quit()

    
df = pd.DataFrame(data)
df.set_index("publication",inplace=True)
df

orchestrator.py - line 45 - INFO - Data fetcher setup completed.
2951215865.py - line 26 - INFO - Processing URL: https://www.ncbi.nlm.nih.gov/pubmed/39486496
data_fetcher.py - line 65 - INFO - Non-API URL detected, or API unsupported. Webscraper update


Processing URL 389.
Time elapsed: 2.1852731704711914
3 links found for dataset PXD056511


2951215865.py - line 34 - INFO - Fetching Raw content
2951215865.py - line 26 - INFO - Processing URL: https://dx.doi.org/10.6019/PXD056511
data_fetcher.py - line 30 - INFO - Publisher: doi
data_fetcher.py - line 65 - INFO - Non-API URL detected, or API unsupported. Webscraper update


id not found in raw data


2951215865.py - line 34 - INFO - Fetching Raw content
orchestrator.py - line 30 - INFO - Previous driver quit.
orchestrator.py - line 45 - INFO - Data fetcher setup completed.
2951215865.py - line 26 - INFO - Processing URL: https://www.ncbi.nlm.nih.gov/pubmed/38617292
data_fetcher.py - line 65 - INFO - Non-API URL detected, or API unsupported. Webscraper update


...

Processing URL 441.
Time elapsed: 820.1276791095734
1 links found for dataset PXD052807


2951215865.py - line 34 - INFO - Fetching Raw content


id not found in raw data


orchestrator.py - line 30 - INFO - Previous driver quit.


KeyboardInterrupt: 

In [9]:
df = pd.DataFrame(data)
df.set_index("publication",inplace=True)
df

,dataset_uid,repo_name,raw_html
publication,,,
https://dx.doi.org/10.6019/PXD056511,PXD056511,PRIDE,"<html xmlns=""http://www.w3.org/1999/xhtml"" lan..."
https://dx.doi.org/10.6019/PXD054686,PXD054686,PRIDE,"<html xmlns=""http://www.w3.org/1999/xhtml"" lan..."
https://dx.doi.org/10.26508/LSA.202403007,PXD058295,PRIDE,"<html lang=""en"" dir=""ltr"" xmlns=""http://www.w3..."
https://dx.doi.org/10.3389/FMED.2024.1302637,PXD045874,PRIDE,"<html lang=""en"" data-n-head=""%7B%22lang%22:%7B..."
https://dx.doi.org/10.6019/PXD054390,PXD054390,PRIDE,"<html xmlns=""http://www.w3.org/1999/xhtml"" lan..."
https://dx.doi.org/10.6019/PXD054428,PXD054428,PRIDE,"<html xmlns=""http://www.w3.org/1999/xhtml"" lan..."
https://dx.doi.org/10.1016/j.bioactmat.2024.08.037,PXD050584,PRIDE,"<html lang=""en-US"" class=""toolbar-stuck""><head..."
https://dx.doi.org/10.1002/1878-0261.13654,PXD048538,PRIDE,"<html lang=""en"" class=""pb-page"" data-request-i..."
https://dx.doi.org/10.6019/PXD048514,PXD048514,PRIDE,"<html xmlns=""http://www.w3.org/1999/xhtml"" lan..."


In [10]:
df_old = pd.read_csv("exp_input/raw_data.csv", index_col="publication")
print(df_old.shape)
# append rows to the existing CSV
df_new = pd.concat([df_old,df])
df_new.to_csv("exp_input/PRIDEid_HTML_data.csv")

(220, 3)


In [12]:
df_new.shape

(242, 3)

In [17]:
print(len(df_new.index))
print(len(df_new.index.unique()))

242
221


In [19]:
df_merged = (
    df_new.reset_index()
    .groupby('publication')
    .agg({
        'dataset_uid': lambda x: ','.join(sorted(set(x))),  # Concatenate unique dataset_uids
        'repo_name': lambda x: ','.join(sorted(set(x))),   # Concatenate unique repo_names
        'raw_html': 'first'  # Keep the first raw_html
    })
)

In [ ]:
df_merged.to_csv("exp_input/raw_data.csv")

In [24]:
df_merged

,dataset_uid,repo_name,raw_html
publication,,,
https://dx.doi.org/10.1002/1878-0261.13654,PXD048538,PRIDE,"<html lang=""en"" class=""pb-page"" data-request-i..."
https://dx.doi.org/10.1002/ADHM.202404465,PXD052728,PRIDE,"<html lang=""en"" class=""pb-page"" data-request-i..."
https://dx.doi.org/10.1002/CBIC.202400831,PXD055649,PRIDE,"<html lang=""en"" class=""pb-page"" data-request-i..."
https://dx.doi.org/10.1002/PRCA.202300107,PXD028078,PRIDE,"<html lang=""en"" class=""pb-page"" data-request-i..."
https://dx.doi.org/10.1002/anie.202420149,"PXD056865,PXD057925,PXD058045",PRIDE,"<html lang=""en"" class=""pb-page"" data-request-i..."
...,...,...,...
https://dx.doi.org/10.6019/PXD059932,PXD059932,PRIDE,"<html xmlns=""http://www.w3.org/1999/xhtml"" lan..."
https://dx.doi.org/10.7554/ELIFE.86194.2,PXD054648,PRIDE,"<html lang=""en-US""><head><meta charset=""utf-8""..."
https://dx.doi.org/10.7554/ELIFE.86931.2,PXD037489,PRIDE,"<html lang=""en-US""><head><meta charset=""utf-8""..."
